In [4]:
import numpy as np
import cv2
import math
from operator import itemgetter, attrgetter

In [5]:
def display(img):
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
image = cv2.imread('photos\WrongLane1.jpg', flags=cv2.IMREAD_COLOR)
resized = cv2.resize(image, (1280, 720), interpolation = cv2.INTER_AREA)
display(resized)

gray_image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
display(gray_image)

recontrasted = cv2.equalizeHist(gray_image)
display(recontrasted)

img_hsv = cv2.cvtColor(resized, cv2.COLOR_RGB2HSV)
lower_yellow = np.array([20, 100, 100], dtype = 'uint8')
upper_yellow = np.array([30, 255, 255], dtype='uint8')
mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
mask_white = cv2.inRange(gray_image, 200, 255)
mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
mask_yw_image = cv2.bitwise_and(gray_image, mask_yw)
display(mask_yw_image)

kernel_size = 5
gauss_gray = cv2.GaussianBlur(mask_yw_image, (5,5), 0)
display(gauss_gray)

Setting thresholds for yellow and white

Detecting edges with Canny

In [8]:
low_threshold = 50
high_threshold = 150
canny_edges = cv2.Canny(gauss_gray, low_threshold, high_threshold)
display(canny_edges)

Cropping the image

In [9]:
imshape = canny_edges.shape
print(imshape)

roi_image = canny_edges[int(imshape[0]/2):imshape[0]-40, 0:imshape[1]]
display(roi_image)

(720, 1280)


Doing some magic with Hough space

In [10]:
def filterLines(lines):
    filteredLines = []
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        if math.sin(theta) == 0:
            continue
        a = math.cos(theta)/math.sin(theta)
        if -0.5 <= a <= 0.5:
            continue
        if sum(1 for line in filteredLines if abs(line[0][1] - theta) < 0.25) == 0:
             filteredLines.append(lines[i])
    return filteredLines
    

In [11]:
line_image = cv2.cvtColor(roi_image, cv2.COLOR_GRAY2RGB)
rho = 1
theta = np.pi/180
threshold = 10
min_line_len = 200
max_line_gap = 100

while True:
    lines = cv2.HoughLines(roi_image, rho, theta, threshold, min_line_len, max_line_gap)
    print('Found ' + str(len(lines)) + ' lines')
    print('Filtering lines')
    filteredLines = filterLines(lines)
    if len(filteredLines) <= 3:
        break
    threshold += 10


if filteredLines is not None:
    print('After filtering ' + str(len(filteredLines)) + ' lines')
    for i in range(0, len(filteredLines)):
        rho = filteredLines[i][0][0]
        theta = filteredLines[i][0][1]
        print(f'line {i}: rho={rho} theta={theta}')
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        print(f'From point {pt1} to {pt2}')
        cv2.line(line_image, pt1, pt2, (0, 0, 255), 5)
        
imshape = line_image.shape
cv2.line(line_image, (int(imshape[1]/2), 0), (int(imshape[1]/2), imshape[1]), (255, 0, 0), 2)

display(line_image)

Found 14422 lines
Filtering lines
Found 4948 lines
Filtering lines
Found 1944 lines
Filtering lines
Found 864 lines
Filtering lines
Found 438 lines
Filtering lines
Found 277 lines
Filtering lines
Found 173 lines
Filtering lines
After filtering 2 lines
line 0: rho=-529.0 theta=2.583087205886841
From point (-81, -1128) to (978, 567)
line 1: rho=337.0 theta=1.0821040868759155
From point (-724, 767) to (1041, -171)


You can do it

In [12]:
imshape = line_image.shape
y = imshape[0]
cutoffPoint = int(imshape[1]/2)

linesData = []
for i in range(0, len(filteredLines)):
    rho = filteredLines[i][0][0]
    theta = filteredLines[i][0][1]
    a = -1 * math.cos(theta)/math.sin(theta)
    b = rho/math.sin(theta)
    x00 = (y - b)/a
    a *= -1 #convenience
    if (x00 < cutoffPoint and a > 0) or (x00 > cutoffPoint and a < 0):      
        linesData.append((a, x00))
        
linesData = sorted(linesData, key=itemgetter(1))

for i in range(0, len(linesData)):
    print(f'a={linesData[i][0]} x00={linesData[i][1]}')

a=0.5317094949770185 x00=115.99589189609142
a=-1.6003342189973113 x00=823.7436406652968


In [13]:
cutoffPoint = int(imshape[1]/2)
correctSideMsg = 'You are on correct side of the road'
wrongSideMsg = 'CAUTION! You are on the WRONG side! MURDERER!'

linesOnLeft = [l for l in linesData if l[1] < cutoffPoint]
linesOnRight = [l for l in linesData if l[1] > cutoffPoint]
    
if len(linesData) == 3:
    if(len(linesOnLeft) < len(linesOnRight)): #LEFT SIDE
        print(correctSideMsg)
    else: #RIGHT SIDe
        print(wrongSideMsg)
elif len(filteredLines) == 2 and (len(linesOnLeft) == len(linesOnRight)):
    if abs(linesOnLeft[0][0]) > abs(linesOnRight[0][0]): #LEFT SIDE
        print(correctSideMsg)
    else:
        print(wrongSideMsg)

CAUTION! You are on the WRONG side! MURDERER!
